In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import gc
import time
from pandas.core.common import SettingWithCopyWarning
import warnings
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from pandas.io.json import json_normalize
import json
import os
warnings.simplefilter('error', SettingWithCopyWarning)
gc.enable()
%matplotlib inline

In [2]:
def load_df(csv_path='/home/baitong/pywork/RevenuePrediction/all/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
#     print(df.head())
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
path = '/home/baitong/pywork/RevenuePrediction/'

In [4]:
%%time
train = load_df()
test = load_df(csv_path='/home/baitong/pywork/RevenuePrediction/all/test.csv')


Loaded train.csv. Shape: (903653, 55)
Loaded test.csv. Shape: (804684, 53)
CPU times: user 3min 52s, sys: 4.28 s, total: 3min 56s
Wall time: 3min 56s


In [5]:
#获取不变的常量列，模型无法在常量数据计学到东西，数据与处理时需要drop
const_cols = [c for c in train.columns if train[c].nunique(dropna=False)==1 ]
print(const_cols)


['socialEngagementType', 'device_browserSize', 'device_browserVersion', 'device_flashVersion', 'device_language', 'device_mobileDeviceBranding', 'device_mobileDeviceInfo', 'device_mobileDeviceMarketingName', 'device_mobileDeviceModel', 'device_mobileInputSelector', 'device_operatingSystemVersion', 'device_screenColors', 'device_screenResolution', 'geoNetwork_cityId', 'geoNetwork_latitude', 'geoNetwork_longitude', 'geoNetwork_networkLocation', 'totals_visits', 'trafficSource_adwordsClickInfo.criteriaParameters']


In [6]:
train = train.drop(const_cols , axis=1)
test = test.drop(const_cols, axis=1)

In [7]:
train['depth']=train['trafficSource_referralPath'][train['trafficSource_referralPath'].notnull()].apply(lambda x:x.count('/'))
test['depth']=test['trafficSource_referralPath'][test['trafficSource_referralPath'].notnull()].apply(lambda x:x.count('/'))
# train['length']=train['trafficSource_referralPath'][train['trafficSource_referralPath'].notnull()].apply(lambda x:len(x))#例子里好像没有放在特征里
# train['path_complexity']=train['trafficSource_referralPath'][train['trafficSource_referralPath'].notnull()].apply(lambda x:train['depth']/train['length'])

In [8]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device_browser',
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
       'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',
       'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_campaignCode', 'trafficSource_isTrueDirect',
       'trafficSource_keyword', 'trafficSource_medium',
       'trafficSource_referralPath', 'trafficSource_source', 'depth'],


In [9]:
##将缺失值表示统一
def replaceCatNa(df):
    Nalist = ['(not set)','0','not available in demo dataset','unknown.unknown','(not provided)','(none)']
    catlist = ['device_browser','device_operatingSystem','geoNetwork_city','geoNetwork_continent',
              'geoNetwork_metro','geoNetwork_networkDomain','geoNetwork_region','geoNetwork_subContinent',
              'trafficSource_campaign','trafficSource_keyword','trafficSource_medium']
    for col in catlist:
        df[col] = df[col].astype('str') 
        df[col] = df[col].apply(lambda x : np.nan if x in Nalist else x)
    return df

In [10]:
train = replaceCatNa(train)
test = replaceCatNa(test)

In [11]:
# 去除test中不存在的列
train.drop('trafficSource_campaignCode',inplace = True,axis = 1)

In [12]:
train.shape,test.shape

((903653, 36), (804684, 35))

In [13]:
###将时间戳转换为标准时间表示
train.visitStartTime = pd.to_datetime(train.visitStartTime, unit='s')
test.visitStartTime = pd.to_datetime(test.visitStartTime, unit='s')
train["date"] = train.visitStartTime
test["date"] = test.visitStartTime

In [14]:
for df in [train, test]:
    df['weekday'] = df['date'].dt.dayofweek.astype(object)
    df['time'] = df['date'].dt.second + df['date'].dt.minute*60 + df['date'].dt.hour*3600
    df['month'] = df['date'].dt.month   # it must not be included in features during learning!
    df['day'] = df['date'].dt.date       # it must not be included in features during learning!
    df['hours'] = df['date'].dt.hour

In [15]:
####5，6为双休日
train['hours'].value_counts()

18    51096
17    50797
16    49039
15    48092
19    47911
20    46753
14    44490
21    44057
13    40176
22    39584
23    35690
12    34487
0     33482
1     32512
2     32063
3     31235
7     31121
11    30904
8     30889
9     30184
4     29942
10    29869
6     29696
5     29584
Name: hours, dtype: int64

In [16]:
###增加特征 ：是否为双休日
train['isWeekend'] = train['weekday'].apply(lambda x:1 if x>=5 else 0)
test['isWeekend'] = test['weekday'].apply(lambda x:1 if x>=5 else 0)

In [17]:
###填补totals.pageviews值
predmis = train[train['totals_pageviews'].isnull()]['totals_hits'].values
train.loc[(train['totals_pageviews'].isnull()),'totals_pageviews' ] = predmis

predmis = test[test['totals_pageviews'].isnull()]['totals_hits'].values
test.loc[(test['totals_pageviews'].isnull()),'totals_pageviews' ] = predmis

In [18]:
####增加特征：浏览器是否为Chrome
train['isChrome'] = train['device_browser'].apply(lambda x:1 if x=='Chrome' else 0)
test['isChrome'] = test['device_browser'].apply(lambda x:1 if x=='Chrome' else 0)

In [19]:
####增加特征：source是否为youtube ,mail.googleplex.com youtube.com
train['trafficSource_source'].astype('str',inplace = True)
test['trafficSource_source'].astype('str',inplace = True)
train['s_ismall'] = train['trafficSource_source'].apply(lambda x:1 if x=='mall.googleplex.com' else 0)
test['s_ismall'] = test['trafficSource_source'].apply(lambda x:1 if x=='mall.googleplex.com' else 0)

train['s_isyoutube'] = train['trafficSource_source'].apply(lambda x:1 if x=='youtube.com' else 0)
test['s_isyoutube'] = test['trafficSource_source'].apply(lambda x:1 if x=='youtube.com' else 0)

In [20]:
##增加特征：是否为desktop
train['d_isdesktop'] = train['device_deviceCategory'].apply(lambda x:1 if x=='desktop' else 0)
test['d_isdesktop'] = test['device_deviceCategory'].apply(lambda x:1 if x=='desktop' else 0)

train['d_Macintosh'] = train['device_operatingSystem'].apply(lambda x:1 if x=='Macintosh' else 0)
test['d_Macintosh'] = test['device_operatingSystem'].apply(lambda x:1 if x=='Macintosh' else 0)

train['d_Macintosh'] = train['device_operatingSystem'].apply(lambda x:1 if x=='Macintosh' else 0)
test['d_Macintosh'] = test['device_operatingSystem'].apply(lambda x:1 if x=='Macintosh' else 0)

train['d_Linux'] = train['device_operatingSystem'].apply(lambda x:1 if x=='Linux' else 0)
test['d_Linux'] = test['device_operatingSystem'].apply(lambda x:1 if x=='Linux' else 0)

train['d_iOS'] = train['device_operatingSystem'].apply(lambda x:1 if x=='iOS' else 0)
test['d_iOS'] = test['device_operatingSystem'].apply(lambda x:1 if x=='iOS' else 0)

train['d_Android'] = train['device_operatingSystem'].apply(lambda x:1 if x=='Android' else 0)
test['d_Android'] = test['device_operatingSystem'].apply(lambda x:1 if x=='Android' else 0)

train['d_Windows Phone'] = train['device_operatingSystem'].apply(lambda x:1 if x=='Windows Phone' else 0)
test['d_Windows Phone'] = test['device_operatingSystem'].apply(lambda x:1 if x=='Windows Phone' else 0)


In [21]:
train['channel_referral'] = train['channelGrouping'].apply(lambda x:1 if x=='Referral' else 0)
train['channel_Social'] = train['channelGrouping'].apply(lambda x:1 if x=='Social' else 0)
test['channel_referral'] = test['channelGrouping'].apply(lambda x:1 if x=='Referral' else 0)
test['channel_Social'] = test['channelGrouping'].apply(lambda x:1 if x=='Social' else 0)

In [22]:
def dealbrowser(s):
    if s == "Safari "or s == "Firefox":
        return "mainstream"
    elif s == "Chrome":
        return "Chrome"
    else:
        return "others"
train['browser'] = train['device_browser'].apply(dealbrowser)

In [23]:
##增加特征：是否为desktop
train['m_referral'] = train['trafficSource_medium'].apply(lambda x:1 if x=='referral' else 0)
test['m_referral'] = test['trafficSource_medium'].apply(lambda x:1 if x=='referral' else 0)

In [29]:
train['single_visit'] = train['visitNumber'].apply(lambda x:0 if x!=1 else 1)
test['single_visit'] = test['visitNumber'].apply(lambda x:0 if x!=1 else 1)

train['totals_hits'] = train['totals_hits'].astype('float') 
test['totals_hits'] = test['totals_hits'].astype('float') 
train['totals_pageviews'] = train['totals_pageviews'].astype('float') 
test['totals_pageviews'] = test['totals_pageviews'].astype('float') 
train['hits_ratio'] = train['totals_hits']/train['totals_pageviews']
test['hits_ratio'] = test['totals_hits']/test['totals_pageviews']

# tr_te[, single_visit := ifelse(visitNumber == 1,1,0) ]
#     tr_te[, hits_ratio := as.numeric(hits)/as.numeric(pageviews)]
#     tr_te[, domain_site := gsub("^.*\\.","", networkDomain)]
#     tr_te[, adwordsClickInfo.isVideoAd := ifelse(is.na(adwordsClickInfo.isVideoAd), 1, 0)]


In [37]:
##去除离群点'Anguilla'
train =train.drop(index = train['geoNetwork_country'][train['geoNetwork_country']=='Anguilla'].index)

In [44]:
def dealMissingData(df):
    for c in df.columns:
        num_missing = df[c].isnull().sum() 
        if num_missing != 0:
            print("missing col is  ",c,'   missing num is',num_missing)
    print('*'*20)
    num_feature = ["totals_bounces","totals_newVisits",'depth','trafficSource_adwordsClickInfo.page']
    cat_feature = ['trafficSource_adContent','trafficSource_adContent',
                  'trafficSource_campaign','trafficSource_isTrueDirect',
                  'trafficSource_keyword','trafficSource_referralPath',
                  'device_browser','device_operatingSystem','geoNetwork_city',
                  'geoNetwork_continent','geoNetwork_metro','geoNetwork_networkDomain',
                  'geoNetwork_region','geoNetwork_subContinent','trafficSource_adwordsClickInfo.adNetworkType',
                  'trafficSource_adwordsClickInfo.isVideoAd','trafficSource_medium','trafficSource_adwordsClickInfo.slot']
    for col in num_feature:
        df[col].fillna(0,inplace=True)
    for col in cat_feature:
        df[col].fillna(0,inplace=True)
    for c in df.columns:
        num_missing = df[c].isnull().sum() 
        if num_missing != 0:
            print("now missing col is  ",c,'   missing num is',num_missing)
    print('---------------done---------------')
    return df

In [45]:
train = dealMissingData(train)
test = dealMissingData(test)

missing col is   totals_transactionRevenue    missing num is 892138
missing col is   trafficSource_adwordsClickInfo.gclId    missing num is 882091
missing col is   trafficSource_adwordsClickInfo.page    missing num is 882192
missing col is   trafficSource_adwordsClickInfo.slot    missing num is 882192
********************
now missing col is   totals_transactionRevenue    missing num is 892138
now missing col is   trafficSource_adwordsClickInfo.gclId    missing num is 882091
---------------done---------------
missing col is   trafficSource_adwordsClickInfo.gclId    missing num is 750822
missing col is   trafficSource_adwordsClickInfo.page    missing num is 750870
missing col is   trafficSource_adwordsClickInfo.slot    missing num is 750870
********************
now missing col is   trafficSource_adwordsClickInfo.gclId    missing num is 750822
---------------done---------------


In [46]:
def clearRare(columnname, limit = 1000):
    # you may search for rare categories in train, train&test, or just test
    #vc = pd.concat([train[columnname], test[columnname]], sort=False).value_counts()
    vc = test[columnname].value_counts()
    
    common = vc > limit
    common = set(common.index[common].values)
    print("Set", sum(vc <= limit), columnname, "categories to 'other';", end=" ")
    
    train.loc[train[columnname].map(lambda x: x not in common), columnname] = 'other'
    test.loc[test[columnname].map(lambda x: x not in common), columnname] = 'other'
    print("now there are", train[columnname].nunique(), "categories in train")

In [47]:
###数量过少的数据无参考价值，统一归为others
clearRare("device_browser")
clearRare("device_operatingSystem")
clearRare("geoNetwork_country")
clearRare("geoNetwork_city")
clearRare("geoNetwork_metro")
clearRare("geoNetwork_networkDomain")
clearRare("geoNetwork_region")
clearRare("geoNetwork_subContinent")
clearRare("trafficSource_adContent")
clearRare("trafficSource_campaign")
clearRare("trafficSource_keyword")
clearRare("trafficSource_medium")
clearRare("trafficSource_referralPath")
clearRare("trafficSource_source")

Set 98 device_browser categories to 'other'; now there are 11 categories in train
Set 15 device_operatingSystem categories to 'other'; now there are 8 categories in train
Set 160 geoNetwork_country categories to 'other'; now there are 60 categories in train
Set 656 geoNetwork_city categories to 'other'; now there are 76 categories in train
Set 86 geoNetwork_metro categories to 'other'; now there are 23 categories in train
Set 25689 geoNetwork_networkDomain categories to 'other'; now there are 61 categories in train
Set 314 geoNetwork_region categories to 'other'; now there are 61 categories in train
Set 5 geoNetwork_subContinent categories to 'other'; now there are 19 categories in train
Set 46 trafficSource_adContent categories to 'other'; now there are 4 categories in train
Set 23 trafficSource_campaign categories to 'other'; now there are 5 categories in train
Set 2409 trafficSource_keyword categories to 'other'; now there are 8 categories in train
Set 0 trafficSource_medium categor

In [48]:
train.set_index("visitStartTime", inplace=True)
test.set_index("visitStartTime", inplace=True)
train.sort_index(inplace=True)
test.sort_index(inplace=True)

In [49]:
###增加上一次会话和下一次会话的时间间隔，时间间隔越短说明用户购买商品欲望更强烈
df = pd.concat([train, test])
df.sort_values(['fullVisitorId', 'date'], ascending=True, inplace=True)
df['prev_session'] = (df['date'] - df[['fullVisitorId', 'date']].groupby('fullVisitorId')['date'].shift(1)).astype(np.int64) // 1e9 // 60 // 60
df['next_session'] = (df['date'] - df[['fullVisitorId', 'date']].groupby('fullVisitorId')['date'].shift(-1)).astype(np.int64) // 1e9 // 60 // 60
df.sort_index(inplace=True)

train = df[:len(train)]
test = df[len(train):]

/home/baitong/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [57]:
##将缺失的0 转换为字符串格式
str_feature = ['trafficSource_adContent','trafficSource_campaign','trafficSource_isTrueDirect',
              'trafficSource_keyword','trafficSource_referralPath',
              'device_browser','device_operatingSystem','geoNetwork_city',
              'geoNetwork_continent','geoNetwork_metro','geoNetwork_networkDomain',
              'geoNetwork_region','geoNetwork_subContinent','trafficSource_adwordsClickInfo.adNetworkType',
              'trafficSource_adwordsClickInfo.isVideoAd','trafficSource_medium','trafficSource_adwordsClickInfo.slot']
for df in [train, test]:
    for col in str_feature:
        df[col] = df[col].astype('str')

In [58]:
##组合特征
for df in [train, test]:
    df['source_country'] = df['trafficSource_source'] + '_' + df['geoNetwork_country']
    df['campaign_medium'] = df['trafficSource_campaign'] + '_' + df['trafficSource_medium']
    df['browser_category'] = df['device_browser'] + '_' + df['device_deviceCategory']
    df['browser_os'] = df['device_browser'] + '_' + df['device_operatingSystem']

In [61]:
##组合特征
for df in [train, test]:
    df['device_deviceCategory_channelGrouping'] = df['device_deviceCategory'] + "_" + df['channelGrouping']
    df['channelGrouping_browser'] = df['device_browser'] + "_" + df['channelGrouping']
    df['channelGrouping_OS'] = df['device_operatingSystem'] + "_" + df['channelGrouping']
    
    for i in ['geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country','geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region','geoNetwork_subContinent']:
        for j in ['device_browser','device_deviceCategory', 'device_operatingSystem', 'trafficSource_source']:
            df[i + "_" + j] = df[i] + "_" + df[j]
    
    df['content_source'] = df['trafficSource_adContent'].astype(str) + "_" + df['source_country']
    df['medium_source'] = df['trafficSource_medium'] + "_" + df['source_country']

In [62]:
##针对用户增加点击浏览量均值特征，以及访问人数最大值特征
for feature in ["totals_hits", "totals_pageviews"]:
    info = pd.concat([train, test], sort=False).groupby("fullVisitorId")[feature].mean()
    train["usermean_" + feature] = train.fullVisitorId.map(info)
    test["usermean_" + feature] = test.fullVisitorId.map(info)
    
for feature in ["visitNumber"]:
    info = pd.concat([train, test], sort=False).groupby("fullVisitorId")[feature].max()
    train["usermax_" + feature] = train.fullVisitorId.map(info)
    test["usermax_" + feature] = test.fullVisitorId.map(info)

In [65]:
excluded = ['date', 'fullVisitorId', 'sessionId', 'totals_transactionRevenue', 'visitId', 'visitStartTime', 
            'month', 'day', 'help','trafficSource_adwordsClickInfo.gclId']

cat_cols = [f for f in train.columns if (train[f].dtype == 'object' and f not in excluded)]
real_cols = [f for f in train.columns if (not f in cat_cols and f not in excluded)]
print("cat_cols num: ",len(cat_cols))
print("cat_cols: ",cat_cols)
print("real_cols num: ",len(real_cols))
print("real_cols: ",real_cols)

cat_cols num:  63
cat_cols:  ['browser', 'channelGrouping', 'device_browser', 'device_deviceCategory', 'device_operatingSystem', 'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region', 'geoNetwork_subContinent', 'totals_bounces', 'totals_newVisits', 'trafficSource_adContent', 'trafficSource_adwordsClickInfo.adNetworkType', 'trafficSource_adwordsClickInfo.isVideoAd', 'trafficSource_adwordsClickInfo.page', 'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign', 'trafficSource_isTrueDirect', 'trafficSource_keyword', 'trafficSource_medium', 'trafficSource_referralPath', 'trafficSource_source', 'weekday', 'source_country', 'campaign_medium', 'browser_category', 'browser_os', 'device_deviceCategory_channelGrouping', 'channelGrouping_browser', 'channelGrouping_OS', 'geoNetwork_city_device_browser', 'geoNetwork_city_device_deviceCategory', 'geoNetwork_city_device_operatingSystem', 'geoNetwork_city_trafficSou

In [66]:
y_reg = train['totals_transactionRevenue']

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    test[col] = lbl.transform(list(test[col].values.astype('str')))